In [3]:
import os
import sys
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

/Users/dummy/Downloads/Langchain Projects/1-QnA Chatbot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [19]:
openai = OpenAI()
claude = anthropic.Anthropic()

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [9]:
system_message = 'You are an assistant would would replace Python code with C++ code'
system_message+= 'Respond only with C++ code.Use comments sparingly.'
system_message+='C++ code needs to be identical in performance and functionality'


In [10]:
def user_prompt_for(python):
    user_prompt ='Rewrite python code in the fastest possible way to produce identical output.'
    user_prompt+= 'Respond only with C++ code'
    user_prompt+='pay attendtion to the number types to ensure no int overflow'
    user_prompt+=python
    return user_prompt

In [11]:
def messages_for(python):
    return [
        {'role':'system','content':system_message},
        {'role':'user','content':user_prompt_for(python)}
    ]

In [12]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [20]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [14]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=500,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [15]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [16]:
exec(pi)

Result: 3.141592658589
Execution Time: 8.831092 seconds


In [21]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <chrono>
#include <iomanip>

double calculate(long long iterations, double param1, double param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000LL, 4.0, 1.0) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();
    
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;

    std::chrono::duration<double> execution_time = end_time - start_time;
    std::cout << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```